In [61]:
import pandas as pd
#import lux
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import precision_score,f1_score,recall_score


In [35]:
# read the data from a CSV file (included in the repository)
df = pd.read_csv("data/train.csv")

In [37]:
# 1. Remove the columns "Name" and "PassengerId" (because we know they are irrelevant for our problem).
df.pop('Name')
df.pop('PassengerId')


0        1
1        2
2        3
3        4
4        5
      ... 
886    887
887    888
888    889
889    890
890    891
Name: PassengerId, Length: 891, dtype: int64

In [39]:
# 2. Convert all non-numeric columns into numeric ones. The non-numeric columns are "Sex" x, "Cabin" x, "Ticket" and "Embarked" x.
N = len(df['Sex'])
for i in range(N):

    s = df['Sex'][i]
    if  s == 'male':
        df['Sex'][i] = 0
    elif s == 'female':
        df['Sex'][i] = 1
    else:
        None 

    e = df['Embarked'][i]
    if  e == 'S':
        df['Embarked'][i] = 19
    elif e == 'C':
        df['Embarked'][i] = 3
    elif e == 'Q':
        df['Embarked'][i] = 17     
    else:
        None

    c = df['Cabin'][i] #if cabin is known, 1, else 0

    if type(c) == str:
        df['Cabin'][i] = 1
    else:
        df['Cabin'][i] = 0    

C:\Users\Jan\AppData\Local\Temp\ipykernel_14628\2489652324.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sex'][i] = 0
C:\Users\Jan\AppData\Local\Temp\ipykernel_14628\2489652324.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Embarked'][i] = 19
C:\Users\Jan\AppData\Local\Temp\ipykernel_14628\2489652324.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Cabin'][i] = 0
C:\Users\Jan\AppData\Local\Temp\ipykernel_146

In [40]:
df.pop('Ticket')

0             A/5 21171
1              PC 17599
2      STON/O2. 3101282
3                113803
4                373450
             ...       
886              211536
887              112053
888          W./C. 6607
889              111369
890              370376
Name: Ticket, Length: 891, dtype: object

In [49]:
# 3. Remove all rows that contain missing values
df=df.dropna()

# Step 4

In [50]:
# 1. As a next step, we need to split the input features from the training labels. This can be done easily with `pandas`.
features = ['Pclass','Sex','Age','SibSp','Parch','Fare','Cabin','Embarked']
target = ['Survived']

In [44]:
# 2. Secondly, we need to split training and test data. This can be done with the function [`sklearn.model_selection.train_test_split()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split) from the `scikit-learn` library.

def TrainTestSplit(D,features,split = 0.2):
    T = int(len(D)*split) #Länge des Testanteils

    return [df[features][:T],df[target][:T]],[df[features][T:],df[target][T:]] #Erst Test, dann Trainingsintervall


In [45]:
# 3. Finally, initialize a LogisticRegression object with a `liblinear` solver, and fit it to the training data.

def fit_model(x_train,y_train,x_test,y_test,regrtype= linear_model.LogisticRegression(penalty='none')):
    if type(x_train)== pd.core.frame.DataFrame:
        x_train = np.array(x_train)
        y_train = np.array(y_train)
        x_test = np.array(x_test)
        y_test = np.array(y_test)

    
    model = regrtype
    model.fit(x_train,y_train)
    y_pred= model.predict(x_test)
    return model.coef_,y_pred


In [51]:

data_train,data_test= TrainTestSplit(df,features)


In [57]:
coef,y_pred= fit_model(data_train[0],data_train[1],data_test[0],data_test[1])
y_test = data_test[1]

c:\Users\Jan\miniconda3\envs\normal\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Jan\miniconda3\envs\normal\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [64]:
# 4. Lastly, calculate precision/recall/f-score on the test data using the appropriate functions from `scikit-learn`.
print('precision score:', precision_score(y_test,y_pred),'\n', 'f1 score :', f1_score(y_test,y_pred),'\n','recall score:', recall_score(y_test,y_pred))


precision score: 0.7978723404255319 
 f1 score : 0.6944444444444443 
 recall score: 0.6147540983606558
